![dio](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRGM4HStCQ9njfsBLY61heiJKjA__o-fgU1pxlrpEwnZA4JJnwvu6s8MfN2C_c7v62DTnQ)

# Desafio: Criando um sistema bancário com python

## Objetivo Geral: Criar um sistema bancário com as operações: sacar, depositar e visualizar extrato.

## Regras:
* **Operação de Depósito:** Deve ser possível depositar valores positivos para a minha conta bancária. Todos os depósitos devem ser armazenados em uma variável e exibidos na operação de extrato.


* **Operação de Saque:** O sistema deve permitir realizar 3 saques diários com limite máximo de R$ 500,00 por saque. Caso o usuário não tenha saldo em conta, o sistema deve exibir uma mensagem informando que não será possível sacar o dinheiro por falta de saldo.


* **Operação de Extrato:**Essa operação deve listar todos os depósitos e saques realizados na conta. No fim da listagem deve ser exibido o saldo atual da conta. Se o extrato estiver em branco, exibir a mensagem: "Não foram realizadas movimentações."

In [61]:
# Menu Inicial
def pagina_inicial():
  opcoes = ['f','c','q']


  while True:
    menu_inicial = f'''\t\t\t####### PÁGINA INICIAL #######

                       Olá, Bem vindo ao Wanabank!

                        O que deseja fazer?

                        [f]\tFazer Login.
                        [c]\tCriar Conta de Usuário.
                        [q]\tSair.

                       => '''
    login = input(menu_inicial).casefold().strip()[0]

    if login in opcoes:
      return login

    else:
      ope_inv = ' Opção inválida! Tente novamente! '
      print('\n',ope_inv.center(40, "#"),'\n')

In [62]:
#Criar Conta de usuário
def inserir_cpf():
  while True:
    cpf = input('\nInsira seu CPF (Use apenas números): ').strip()

    if not cpf_valido(cpf):
      print('\nCPF inválido! Digite novamente!\n')
      continue

    if cpf_cadastrado(cpf, usuarios):
      print('O CPF informado já está cadastrado no sistema!')
      continue

    break

  return cpf


def cpf_valido(cpf):
    cpf = cpf.replace('-', '').replace('.', '')
    return cpf.isnumeric() and len(cpf) == 11


def cpf_cadastrado(cpf, usuarios):
    return cpf in usuarios.keys()


def validar_senha(senha):
  confirmar_senha = input('Confirme sua senha: ').strip()
  return confirmar_senha == senha


def criar_senha():
  while True:
    senha = input('\nCrie uma senha: ').strip()

    if not validar_senha(senha):
      print('\nConfirmação inválida! Crie uma nova senha e depois confirme novamente!\n')
      continue

    break

  return senha


def criar_conta(usuarios):

    print('Para continuar o processo de criação de conta, vamos precisar que você nos informe alguns dados.\n')

    nome = input('Informe seu nome: ').strip().title()
    data_nascimento = input('\nData de nascimento no formato dia/mês/ano: ').replace('-', '/').strip()

    cpf = inserir_cpf()

    print('\n*** Informe seu endereço. ***\n')

    logradouro = input('Logradouro: ').strip()
    nro = input('Número: ').strip()
    bairro = input('Bairro: ').strip().title()
    cidade = input('Cidade: ').strip().title()
    uf = input('Sigla do Estado (UF): ').strip().upper()

    senha = criar_senha()

    usuarios[cpf] = {
        'nome': nome,
        'senha': senha,
        'data_nascimento': data_nascimento,
        'endereco': {
            'logradouro': logradouro,
            'n°': nro,
            'bairro': bairro,
            'cidade': cidade,
            'uf': uf
        }
    }

    return "\nConta de Usuário Criada com Sucesso! Você já pode entrar na sua conta.\n"

In [63]:
# Fazer login
def fazer_login(usuarios):
  print('### Insira suas credenciais de acesso ###\n')

  while True:

    cpf_user = input('CPF:\n').strip()

    if not cpf_valido(cpf_user):
      print('\n*CPF com formato inválido, tente novamente!\n')
      continue

    senha_login = input('Senha:\n').strip()

    if cpf_user in list(usuarios.keys()):
      if senha_login == usuarios[cpf_user]['senha']:
        print('\nLogin Efetuado com Sucesso!')
        nome = usuarios[cpf_user]['nome']
        return True, nome

      else:
        print('\nCredenciais de login incorretas. Tente novamente.')
        continue

    else:
      print('\nInformações de login não encontradas.')

      tentar_novamente = input('Deseja tentar novamente?\n\n[s] Sim\n[n] Não\n\n=>').casefold().strip()[0]
      if tentar_novamente == 's':
        continue
      else:
        return False,None

In [64]:
# Criar Conta Corrente

def criar_conta_corrente(usuarios, numero_conta, contas, agencia):
    TIPO_CONTA = 'Corrente'
    decisao = input('\nVocê deseja criar uma nova conta corrente?\n\n[s]\tSim\n[n]\tNão\n\n=>').casefold().strip()

    if decisao == 's':
        while True:

          cpf = input('Informe o seu CPF: ').strip()

          if not cpf_valido(cpf):
            print('\n*CPF com formato inválido, tente novamente!\n')
            continue

          if cpf_cadastrado(cpf, usuarios):
            print('Conta Criada com Sucesso!\n')
            conta_criada = {}
            conta_criada[cpf] = {'usuario': nome, 'agencia': AGENCIA, 'numero_conta': numero_conta, 'tipo_conta': TIPO_CONTA}
            return conta_criada

          else:
            print("\n@@@ Usuário não encontrado, fluxo de criação de conta encerrado! @@@")
            return None

    else:
      print('Tudo bem. Talvez mais tarde.')
      return None

In [128]:
#Função principal

import time
import textwrap

def linha():
    linha = '*'
    return linha * 32

# Função Exibir Menu após o usuário entrar na conta
def exibir_menu(saldo, nome):
    menu = f"""\n\tCONTA BANCÁRIA
    {linha()}
    Cliente: {nome}
    Saldo Atual: R$ {saldo:,.2f}
    {linha()}
    Aperte a tecla correspondente
    a operação que deseja realizar:

    [d]\tDepositar
    [s]\tSacar
    [e]\tExtrato
    [nc] Nova Conta
    [lc] Listar Contas
    [q]\tSair

    => """
    return menu

# Função Depositar
def depositar(valor, saldo, extrato, day, mes, ano, hora_atual):

  if valor <= 0:
    print('\n*ERROR: O VALOR DO DEPÓSITO DEVE SER MAIOR QUE R$ 0.00\nTente novamente!')

  else:

    print(f'OPERAÇÃO REALIZADA COM SUCESSO!\n{linha()}')
    extrato += f'''\nData:{day}/{mes}/{ano} Hora:{hora_atual[10:-5]}\n->DEPÓSITO: R${valor:,.2f}\n'''
    saldo += valor

  return saldo, extrato

# Função de Saque
def sacar(*,saldo, valor, extrato, num_saques, maximo_saque, limite_saque, day, mes, ano, hora_atual ):

  excedeu_saldo = valor > saldo
  excedeu_limite = valor > limite_saque
  excedeu_saques = num_saques >= maximo_saque

  if excedeu_saques:
    print(f'''\n*OPERAÇÃO INVÁLIDA: LIMITE diário de saques = 3. | Saques Realizados hoje = {num_saques}
         => Número máximo de saques excedido.''')

  elif excedeu_limite:
    print(f'''\n*OPERAÇÃO INVÁLIDA: LIMITE por saque = R$ 500.00 | Tentativa de saque registrada: R$ {valor}
          => O valor do saque excede o limite.''')

  elif excedeu_saldo:
    print(f'''\n*OPERAÇÃO INVÁLIDA: O valor do saque excedeu o valor do saldo. Saldo:R$ {saldo}| tentativa de saque:R$ {valor}
        => Você não tem saldo suficiente.''')

  elif (valor >0):
    num_saques += 1
    saldo -= valor
    print(f'OPERAÇÃO REALIZADA COM SUCESSO!\n{linha()},{num_saques}')
    extrato += f'''\nData:{day}/{mes}/{ano} Hora:{hora_atual[10:-5]}\n->SAQUE: R${valor:,.2f}\n'''

  else:
      print(f'''\n*OPERAÇÃO INVÁLIDA: Valor do saque deve ser maior que R$ 0.00 | Valores negativos ou nulos não serão processados pelo sistema!
        => Valor informado é inválido.''')

  return saldo, extrato, num_saques

# Função para exibir o extrato das operações realizadas
def exibir_extrato(saldo, /, *, extrato):
    print("\n================ EXTRATO ================")
    print("Não foram realizadas movimentações." if not extrato else extrato)
    print(f"\nSaldo:\t\tR$ {saldo:.2f}")
    print("==========================================")


# Função para listar as contas do usuário
def listar_contas(contas, usuarios):
    cpf_usuario_logado = input('Informe seu CPF: ').strip()

    if not cpf_valido(cpf_usuario_logado):
        return '\n*CPF com formato inválido, operação cancelada!\n'

    if not cpf_cadastrado(cpf_usuario_logado, usuarios):
        return '\n*CPF não cadastrado, operação cancelada!\n'

    contas_usuario = contas[0].get(cpf_usuario_logado)

    if contas_usuario == '':
        return f'\nO usuário com CPF {cpf_usuario_logado} não possui contas cadastradas.\n'

    print(f'\nContas do usuário com CPF {cpf_usuario_logado}:\n')

    for conta in contas:
        linha = f"""\
            Agência:\t{conta[cpf_usuario_logado]['agencia']}
            C/C:\t\t{conta[cpf_usuario_logado]['numero_conta']}
            Titular:\t{conta[cpf_usuario_logado]['usuario']}
        """
        print("=" * 100)
        print(textwrap.dedent(linha))

usuarios = {}
contas = []
extrato = ""
saldo = 0
logado = False
num_saques = 0
MAX_SAQUES_DIARIO = 3
LIMITE_SAQUE = 500

# Código do sistema
while True:

  hora_atual = time.ctime()
  day = time.gmtime().tm_mday
  mes = time.gmtime().tm_mon
  ano = time.gmtime().tm_year
  AGENCIA = '0001'


  if not logado:
    opcao = pagina_inicial()

    if opcao == 'q':
      print("Obrigado por usar o Wanabank! Até breve!")
      break

    elif opcao == 'f':
      logado, nome = fazer_login(usuarios)

      while logado:
        escolha = input(exibir_menu(saldo, nome)).strip().casefold()

        if escolha == 'd':
          valor = float(input('Insira o valor que deseja Depositar: R$'))
          saldo, extrato = depositar(valor, saldo, extrato,day, mes, ano, hora_atual)
          continue

        elif escolha == 's':
          valor_saque = float(input('Insira o valor que deseja Sacar: R$'))
          saldo, extrato, num_saques = sacar(valor = valor_saque, saldo = saldo, extrato = extrato,
                                day = day, mes = mes, ano = ano, hora_atual = hora_atual,
                                num_saques = num_saques, maximo_saque = MAX_SAQUES_DIARIO, limite_saque = LIMITE_SAQUE)

          continue

        elif escolha == 'e':
          exibir_extrato(saldo, extrato=extrato)
          continue

        elif escolha == 'nc':
          numero_conta = len(contas) + 1
          conta = criar_conta_corrente(usuarios, numero_conta, contas, agencia = AGENCIA)

          if conta:
            contas.append(conta)
          continue

        elif escolha == 'lc':
          listar_contas(contas, usuarios)
          continue

        elif escolha == 'q':
          print(f'\nObrigado por usar nossos serviços. Até logo, {nome}!\n')
          logado = False

      else:
        continue

    elif opcao == 'c':
      resultado = criar_conta(usuarios)
      print(resultado)

    else:
      print('Opção inválida, escolha uma opção válida!')


			####### PÁGINA INICIAL #######

                       Olá, Bem vindo ao Wanabank!

                        O que deseja fazer?

                        [f]	Fazer Login.
                        [c]	Criar Conta de Usuário.
                        [q]	Sair.

                       => c
Para continuar o processo de criação de conta, vamos precisar que você nos informe alguns dados.

Informe seu nome: wandersen

Data de nascimento no formato dia/mês/ano: 10/01/1945

Insira seu CPF (Use apenas números): 00000000000

*** Informe seu endereço. ***

Logradouro: rua tal 
Número: 04
Bairro: europa
Cidade: londrina
Sigla do Estado (UF): ce

Crie uma senha: 010203
Confirme sua senha: 010203

Conta de Usuário Criada com Sucesso! Você já pode entrar na sua conta.

			####### PÁGINA INICIAL #######

                       Olá, Bem vindo ao Wanabank!

                        O que deseja fazer?

                        [f]	Fazer Login.
                        [c]	Criar Conta de Usuário.
           

**OBS**: Caso deseje salvar os valores do dicionário: 'usuário' ou da lista de contas, basta comentar os valores após utilizar o sistema pela primeira vez.

Ex:

```
#usuarios = { }
#contas = [ ] #e se quiser salvar o valor do saldo, comenta também:
#saldo = 0 após também usar pela primeira vez.
```

